<a id='1_tutorial'></a>
# Tutorial 1

In this tutorial we'll cover the first steps of PyMica including preparation of weather station data and explanatory variable fields.

   - [1.1 Preparation of weather station data](#prep_aws_data)  
   - [1.2 Generate explanatory data field](#gen_explanatory)  

<a id='prep_aws_data'></a>
### 1.1 Preparation of weather station data

The data format of weather stations data used by PyMica is a list containing a dictionary for each weather station including, at least, the following vairables:

   - Longitude
   - Latitude
   - Identification code
   - Meteorological variable value to interpolate
   - Value of the explanatory variables that will be used in the multiple linear regression calculation (altitude, distance to coast...)
  
An element of the list containing this variables are organized as follows for each weather station:

```json
       {'id': value, 
        'temp': value,
        'lon': value,
        'lat': value,
        'altitude': value,
        ...}
```
The weather station data is supplied to PyMica in a .json file including a list containing information of all weather stations to consider. 

We'll work with data from the Automatic Weather Station Network (XEMA) of the Meteorological Service of Catalonia. Still, you can provide your own data to PyMica! 

First, we'll import the required libraries.

In [1]:
import os
os.chdir('../../../')

In [1]:
import json
import pandas as pd
from datetime import datetime

Now, let's suppose that our data is in a .csv format. In /cat directory we'll find data from XEMA network for 2017/02/21 12:00 UTC and its corresponding metadata. There are four files, two in .csv format and two in .json format ready to be used by PyMICA. If you want to skip the preparation of .json files, just go to [Section 1.2](#gen_explanatory).

We'll open both .csv files: XEMA_20170221_1200.csv and XEMA_metadata using pandas.

In [2]:
file_data = './data/data/XEMA_20170221_1200.csv'
file_metadata = './data/data/XEMA_metadata.csv'

station_data = pd.read_csv(file_data)
metadata = pd.read_csv(file_metadata)

data = []
for key in station_data['key']:
    df_data = station_data[station_data['key'] == key]
    df_meta = metadata[metadata['key'] == key]
    data.append({'id': key, 
                 'temp': float(df_data['temp']),
                 'lon': float(df_meta['lon']),
                 'lat': float(df_meta['lat']),
                 'altitude': float(df_meta['altitude']),
                 'dist': float(df_meta['dist'])})

FileNotFoundError: File b'./data/data/XEMA_20170221_1200.csv' does not exist

If we print the first element of ```data``` we can see all the required variables for a station (longitude, latitude, distance to coast line, value of temperature, identification code and altitude):

In [ ]:
print('Sample data: ' + str(data[0]))
print('Number of stations: ' + str(len(data)))

Now, we'll save this data into a .json file to use it later.

In [ ]:
with open('./docs/notebooks/tutorials/cat/data/smc_data.json', 'w') as outfile:
    json.dump(data, outfile)

It is also important to creata a file with station metadata information. Then, we'll use ```metadata``` variable to build a .json file.

In [ ]:
values = []
for key in metadata['key']:
    df_meta = metadata[metadata['key'] == key]
    values.append({'id': key,
                   'alt': float(df_meta['altitude']),
                   'lon': float(df_meta['lon']),
                   'lat': float(df_meta['lat']),
                   'dist': float(df_meta['dist'])
                  })

Now, we'll save this data into a .json file to use it later.

In [ ]:
with open('./docs/notebooks/tutorials/cat/data/smc_metadata.json', 'w') as outfile:
    json.dump(values, outfile) 

As you can see, we have introduced distance to coast values in our metadata .json file and in the .json field containing temperature data. The reason is that in the present case, we'll consider distance to coast as a coefficient in the Multiple Linear Regression models. If you want your distance to coast values for your stations or know how they are obtained, please see [Section 3](../preprocessing/pre_processing.ipynb#explanatory_data) from Pre Processing notebook.

This is for the Catalan region, but you can try it with data from any other region!

We have finished the preparation of weather station data for PyMica!

<a id='gen_explanatory'></a>
### 1.2 Generate explanatory data fields

The explanatory variables considered in this tutorial are altitude and distance to coast. Then, we need a Digital Elevation Model (DEM) and a raster matrix of distance to coast values.

In the cat/explanatory directory there is a DEM of Catalonia available and we'll use it for this tutorial. All the explanatory variable raster matrices must have the same spatial resolution and extension. Then, the DEM will be used as a reference to build the others, in this case only the distance to coast raster.

First, we'll import the necessary libraries.

In [ ]:
from osgeo import gdal, osr
from distance.distance_to_sea import get_dist_array
from utils.tiff import get_tif_from_array

For ```get_dist_array``` function we need four parameters: projection, geotransform, size and a coast line file. We'll get the first three from the DEM and the coast line from explanatory folder.

In [ ]:
dem_file = './docs/notebooks/tutorials/cat/explanatory/cat_dem_25831.tif'
dem = gdal.Open(dem_file)

projection = 25831
geotransform = dem.GetGeoTransform()
size = [dem.RasterXSize, dem.RasterYSize]
coast_line = './docs/notebooks/tutorials/cat/explanatory/cat_coast_line.json'

Once all the parameters are set, we call the ```get_dist_array``` function.

In [ ]:
d_coast = get_dist_array(proj=projection, geotransform=geotransform, size=size, dist_file=coast_line)

Now we can get a quick look of the d_coast array:

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(d_coast)
plt.show()

And we must save the array into a .tif file to use it later.

In [ ]:
get_tif_from_array(file_path = './docs/notebooks/tutorials/cat/explanatory/cat_distance_coast.tif',
                   data = d_coast,
                   geotransform = geotransform,
                   projection = projection)

#### Now we have all the explanatory variables fields built and we have finished the first tutorial of PyMica!